In [1]:
import pandas as pd
from itertools import repeat
import re
from orangecontrib.bio.ontology import OBOOntology
import logging
%load_ext autoreload
%autoreload 1
%aimport parse_ontology
from parse_ontology import *
pd.set_option('display.max_colwidth', -1)

In [2]:
def enable_logging(): 
    logger = logging.getLogger()
    logger.setLevel(logging.DEBUG)
    fh = logging.FileHandler('data/process_sample_descriptions.log')
    fh.setLevel(logging.DEBUG)
    formatter = logging.Formatter("%(asctime)s - %(name)s - %(levelname)s - %(message)s")
    fh.setFormatter(formatter)
    logger.removeHandler(logger.handlers[0]) # remove the stream handler
    logger.addHandler(fh)

### Read column headers from fantom5 data. 
Read the column headers and extract sample information from it. 

In [3]:
!ls data

annotation_notes.csv	   hg19.cage_peak_phase1and2combined_ann.txt
column_vars.processed.csv  hg19.cage_peak_phase1and2combined_tpm_ann.osc.txt
column_vars.txt		   human_samples_nature13182-s2
f5_expressionset.Rdata	   process_sample_descriptions.log
fantom5_head2000.txt	   samples1829
fantom5-S1.csv		   samples1829_LIBRARY_IDs
fantom5-S1.xls		   samples1829_simplified
ff-phase2-140729.obo	   subtree_sample.obo


In [4]:
!grep "^##ColumnVariables" data/hg19.cage_peak_phase1and2combined_tpm_ann.osc.txt | cut -d"=" -f2 | head

CAGE peak id
short form of the description below. Common descriptions in the long descriptions has been omited
description of the CAGE peak
transcript which 5end is the nearest to the the CAGE peak
entrezgene (genes) id associated with the transcript
hgnc (gene symbol) id associated with the transcript
uniprot (protein) id associated with the transcript
tpm of 293SLAM rinderpest infection, 00hr, biol_rep1.CNhs14406.13541-145H4
tpm of 293SLAM rinderpest infection, 00hr, biol_rep2.CNhs14407.13542-145H5
tpm of 293SLAM rinderpest infection, 00hr, biol_rep3.CNhs14408.13543-145H6
cut: write error: Broken pipe


In [5]:
!grep "^##ColumnVariables" data/hg19.cage_peak_phase1and2combined_tpm_ann.osc.txt | cut -d"=" -f2 | tail -n+8 | head

tpm of 293SLAM rinderpest infection, 00hr, biol_rep1.CNhs14406.13541-145H4
tpm of 293SLAM rinderpest infection, 00hr, biol_rep2.CNhs14407.13542-145H5
tpm of 293SLAM rinderpest infection, 00hr, biol_rep3.CNhs14408.13543-145H6
tpm of 293SLAM rinderpest infection, 06hr, biol_rep1.CNhs14410.13544-145H7
tpm of 293SLAM rinderpest infection, 06hr, biol_rep2.CNhs14411.13545-145H8
tpm of 293SLAM rinderpest infection, 06hr, biol_rep3.CNhs14412.13546-145H9
tpm of 293SLAM rinderpest infection, 12hr, biol_rep1.CNhs14413.13547-145I1
tpm of 293SLAM rinderpest infection, 12hr, biol_rep2.CNhs14414.13548-145I2
tpm of 293SLAM rinderpest infection, 12hr, biol_rep3.CNhs14415.13549-145I3
tpm of 293SLAM rinderpest infection, 24hr, biol_rep1.CNhs14416.13550-145I4
tail: write error: Broken pipe
tail: write error
cut: write error: Broken pipe


In [6]:
!grep "^##ColumnVariables" data/hg19.cage_peak_phase1and2combined_tpm_ann.osc.txt | cut -d"=" -f2 | tail -n+8 > data/column_vars.txt

In [7]:
sample_infos = !cat data/column_vars.txt

In [8]:
sample_infos[:5]

['tpm of 293SLAM rinderpest infection, 00hr, biol_rep1.CNhs14406.13541-145H4',
 'tpm of 293SLAM rinderpest infection, 00hr, biol_rep2.CNhs14407.13542-145H5',
 'tpm of 293SLAM rinderpest infection, 00hr, biol_rep3.CNhs14408.13543-145H6',
 'tpm of 293SLAM rinderpest infection, 06hr, biol_rep1.CNhs14410.13544-145H7',
 'tpm of 293SLAM rinderpest infection, 06hr, biol_rep2.CNhs14411.13545-145H8']

### Retreiving Information from the ontoloty. 

The column headers are difficult to parse (inconsistent commata, etc.). 
We found an ontology on the fantom5 web page. [1]

First, we check, if all the ids from the column headers appear in the ontology. 

[1] http://fantom.gsc.riken.jp/5/datafiles/latest/extra/Ontology/ff-phase2-140729.obo.txt

In [9]:
OBO_ID_REGEX = re.compile(r'CNhs\d+.(\w+)-(\w+)')

In [10]:
for info_line in sample_infos:
    ff_id = "-".join(OBO_ID_REGEX.search(info_line).groups())
    res = !grep {ff_id} data/ff-phase2-140729.obo.txt 
    assert len(res) > 0

that seems to be the case...

#### Try out the Ontology Parser

In [11]:
obo = OBOOntology()
obo.load(open("data/ff-phase2-140729.obo"))

In [12]:
print(obo.term("FF:1394-42H2").tags())

[('id', 'FF:1394-42H2', None, None), ('name', 'lung, neonate N30, rep1', None, None), ('namespace', 'FANTOM5', None, None), ('subset', 'phase1', None, None), ('subset', 'phase2', None, None), ('subset', 'update022', None, None), ('is_a', 'EFO:0002091', None, 'biological replicate'), ('is_a', 'FF:0011489', None, 'mouse lung- neonate N30 sample')]


In [13]:
obo.term("FF:1394-42H2").name

'lung, neonate N30, rep1'

#### Are all 'samples' annotated as some sort of sample? 

In [14]:
sample = "FF:0000001" # most general sample id 
for info_line in sample_infos[:2]: 
    ff_id = "FF:" + "-".join(OBO_ID_REGEX.search(info_line).groups())
    ids = [term.id for term in obo.super_terms(ff_id)]
    assert sample in ids

seems to be the case, too

## Parse the ontology. 
Using the ontology, we at least don't run into massive comma-parsing trouble again. 
I wrote the python module `parse_ontology.py`. It takes care of
* make sure that there is no inconsistent information between ontology and the sample name (aka. `sample_infos`)
* if information is missing in the ontology, complement it with information from the sample name.
* In such a case, write an entry to `annot_notes`, that we can improve the ontology later on. 

#### Replicates
* There are technical and biological replicates. 
* Technical replicates have the same obo_id (`FF:?????-?????`), but different library ids (`CNhs??????`). 
* The library id is unique. 
* We can identify biological replicates from the sample name with the keywords `biol_rep`, `rep` and `donor`. Having different donors is also a way of having biological replicates. 

As this would be redundant information, we would expect no sample to have both 'biol_rep' and 'donor' in the sample name: 

In [15]:
[x for x in sample_infos if get_donor(x) is not None and get_biol_replicate(x) is not None ]

[]

That seems to be the case. 

Let's now do the parsing and store the processed information as `csv` files. 

In [16]:
enable_logging()

In [17]:
annot_notes = []
annotations = [process_sample_description(obo, sample_info, annot_notes) for sample_info in sample_infos]

In [18]:
annotations_df = pd.DataFrame(annotations)
annot_notes_df = pd.DataFrame(annot_notes)

In [19]:
annotations_df.tail()

,biol_rep,donor,lib_id,name,name_orig,obo_id,tech_rep,time
1824,True,None,CNhs11676,"uterus, adult, pool1","tpm of uterus, adult, pool1.CNhs11676.10100-102D1",FF:10100-102D1,False,None
1825,True,donor1,CNhs11763,"uterus, fetal, donor1","tpm of uterus, fetal, donor1.CNhs11763.10055-101H1",FF:10055-101H1,False,None
1826,True,None,CNhs12854,"vagina, adult, rep1","tpm of vagina, adult.CNhs12854.10204-103F6",FF:10204-103F6,False,None
1827,True,None,CNhs12844,"vein, adult, rep1","tpm of vein, adult.CNhs12844.10191-103E2",FF:10191-103E2,False,None
1828,True,None,CNhs11813,"xeroderma pigentosum b cell line:XPL 17, rep1",tpm of xeroderma pigentosum b cell line:XPL 17.CNhs11813.10563-108A5,FF:10563-108A5,False,None


In [20]:
annot_notes_df.tail()

,field_name,lib_id,new_value,obo_id
1024,biol_rep,CNhs14223,donor10258,FF:10370-105G1
1025,tech_rep,CNhs14223,tech_rep1,FF:10370-105G1
1026,biol_rep,CNhs14551,donor10258,FF:10370-105G1
1027,tech_rep,CNhs14551,tech_rep2,FF:10370-105G1
1028,biol_rep,CNhs14084,donor10223,FF:10366-105F6


In [21]:
len(annot_notes_df[annot_notes_df.field_name == "biol_rep"])

278

#### Choosing lib_id as primary key
* I discovered that obo_id is identical for technical replicates. 
* Now we use the lib_id as primary key, which does not have duplicates: 

In [22]:
annotations_df[annotations_df.duplicated("lib_id", keep=False)]

,biol_rep,donor,lib_id,name,name_orig,obo_id,tech_rep,time


### add sample type information from supplementary information
whether the sample is a tissue, a primary celltype, ... is not properly annotated in the ontology, but we can retrieve that value from the supplementary information data table

In [23]:
si_table = pd.read_excel("data/fantom5-S1.xls", sheetname=1)

In [24]:
pd.set_option('display.max_colwidth', 40)
si_table.tail()

,Sample type,species,description,supplier,sample id,Library_id,Catalog number,external URL,lot number,donor(cell lot),...,Q20 mapped tags,fraction under robust DPI,Number of peaks called,Number of 5' EST/cDNA supported peaks,Fraction peaks corresponding to known 5' end,RIKEN Yokohama ethics application,marker check,used for peak calling,used for expression analysis,top 3 most correlated samples
1385,tissue,Mouse,"whole body, embryo E18",In-house isolation at RIKEN OSC,493,CNhs10516,-,NaN,NaN,NaN,...,4248158,0.711147,10066,9914,0.9849,exempt non human,NaN,yes,yes,"whole_body,_embryo_E17.5_(0.75), who..."
1386,tissue,Mouse,"whole body, neonate N00",In-house isolation at RIKEN OSC,657,CNhs10525,-,NaN,NaN,NaN,...,4437032,0.754926,10825,10693,0.987806,exempt non human,NaN,yes,yes,"whole_body,_embryo_E18_(0.74), whole..."
1387,tissue,Mouse,"whole body, neonate N01",In-house isolation at RIKEN OSC,659,CNhs10576,-,NaN,NaN,NaN,...,5650662,0.697480,10990,10741,0.977343,exempt non human,NaN,yes,yes,"whole_body,_neonate_N06_(0.72), whol..."
1388,tissue,Mouse,"whole body, neonate N06",In-house isolation at RIKEN OSC,692,CNhs10515,-,NaN,NaN,NaN,...,4277029,0.796065,10410,10252,0.984822,exempt non human,NaN,yes,yes,"whole_body,_neonate_N10_(0.76), whol..."
1389,tissue,Mouse,"whole body, neonate N10",In-house isolation at RIKEN OSC,761,CNhs10518,-,NaN,NaN,NaN,...,4157140,0.808407,9651,9503,0.984665,exempt non human,NaN,yes,yes,"whole_body,_neonate_N06_(0.76), bone..."


In [25]:
sample_type = []
for lib_id in annotations_df.lib_id:
    if lib_id in si_table.Library_id.values: 
        sample_type.extend(si_table[si_table.Library_id == lib_id]["Sample type"].values)
    else:
        sample_type.append("unknown")
annotations_df["sample_type"] = sample_type

In [26]:
set(annotations_df.sample_type.values)

{'cell line', 'primary cell', 'tissue', 'unknown'}

### Write everything to a csv file

In [27]:
annotations_df.to_csv("data/column_vars.processed.csv")
annot_notes_df.to_csv("data/annotation_notes.csv")